In [27]:
import numpy as np
import pandas as pd

df = pd.read_csv('/Users/yufan/Desktop/sf_fire_deparment_from_3000k.csv')
df['date'] = pd.to_datetime(df['Received DtTm'], format='%m/%d/%Y %I:%M:%S %p')
df['hour'] = df['date'].dt.hour
df['dayOfWeek'] = df['date'].dt.dayofweek

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df = df[['Zipcode of Incident', 'dayOfWeek','Supervisor District','hour', 'Call Type']]
hotkeydf = pd.get_dummies(data=df, columns=['Zipcode of Incident', 'dayOfWeek','Supervisor District'])
hotkeydf['hour'] = df['hour'] # append non hotkey attribute

In [29]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(hotkeydf, test_size=0.2)
# x is feature, y is label
train_x = train.copy().drop('Call Type', 1)
train_y = train[['Call Type']]
test_x = test.copy().drop('Call Type', 1)
test_y = test[['Call Type']]

In [30]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
logisticRegression = LogisticRegression(random_state=1)
randomForest = RandomForestClassifier(random_state=1)
neuralNetwork = MLPClassifier(alpha = 1)
voting = VotingClassifier(estimators=[('lr', logisticRegression), ('rf', randomForest), ('nn', neuralNetwork)],voting='soft')
classifierList = [sgd, logisticRegression, randomForest, neuralNetwork, voting]
classifierNameList = ["SGD", "logisticRegression", "randomForest", "neuralNetwork", "voting"]

In [31]:
le = preprocessing.LabelEncoder()
le.fit_transform(train_y['Call Type'])

def trainModelAndPrintResult(classifer, classifier_name):
    model = classifer.fit(train_x, le.fit_transform(train_y))
    train_score = model.score(train_x, le.transform(train_y))
    test_score = model.score(test_x, le.transform(test_y))
    print("Classification Score for: ", classifier_name)
    print("Trainning score: ", train_score)
    print("Test score: ", test_score)

In [32]:
for i in range(len(classifierList)):
    trainModelAndPrintResult(classifierList[i], classifierNameList[i])

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (

Classification Score for:  SGD
Trainning score:  0.668071476736
Test score:  0.668376918609
Classification Score for:  logisticRegression
Trainning score:  0.675027372933
Test score:  0.675316958533

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Score for:  randomForest
Trainning score:  0.67767620491
Test score:  0.675984591042

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Score for:  neuralNetwork
Trainning score:  0.673575272227
Test score:  0.673914930266

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Score for:  voting
Trainning score:  0.67522098636
Test score:  0.675397074434
